In [ ]:
import torch
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np

In [ ]:
filename = 'ecfp0'
samples_count = '10M'
model_name = f'molberto_{filename}_{samples_count}'

In [ ]:
molecular_properties = ['Molecular Weight', 'Bioactivities', 'AlogP', 'Polar Surface Area', 'CX Acidic pKa', 'CX Basic pKa']

### Upload and Split Dataset

In [ ]:
dataframe = pd.read_csv("data.csv")

In [ ]:
dataframe = dataframe.drop(columns=['Unnamed: 0', 'Smiles', 'ecfp2', 'ecfp3'])

In [ ]:
def preprocess_data_dataset(df, column):
    for row in tqdm(range(len(df))):
        str_ints = eval(df.iloc[row][column])
        str_fingerprint = ' '.join(str_ints)
        df.at[row, column] = str_fingerprint

In [ ]:
preprocess_data_dataset(dataframe, 'ecfp1')

In [ ]:
dataframe

### Dealing with NaNs

In [ ]:
print('Percentage on NaNs:')
dataframe.isna().mean()

In [ ]:
rows_with_nans = dataframe['Molecular Weight'].isna() | \
                 dataframe['Bioactivities'].isna() | \
                 dataframe['AlogP'].isna() | \
                 dataframe['Polar Surface Area'].isna() | \
                 dataframe['CX Acidic pKa'].isna() | \
                 dataframe['CX Basic pKa'].isna()
print(f'Count of rows without NaNs: {dataframe.shape[0] - dataframe.loc[rows_with_nans].shape[0]}')

In [ ]:
# remove 2 last properties to reduce NaN counts
molecular_properties = ['Molecular Weight', 'Bioactivities', 'AlogP', 'Polar Surface Area']
dataframe = dataframe.drop(columns=['CX Acidic pKa', 'CX Basic pKa'])

In [ ]:
# drop NaN's
dataframe = dataframe.dropna().reset_index(drop=True)

In [ ]:
dataframe

### Split Data

In [ ]:
from sklearn.model_selection import train_test_split

train_dataframe, test_dataframe = train_test_split(dataframe, test_size=0.20, random_state=42)
validation_dataframe, test_dataframe = train_test_split(test_dataframe, test_size=0.50, random_state=42)

In [ ]:
# 80% for train, 10% for test, 10 for validation
print(len(train_dataframe), len(test_dataframe), len(validation_dataframe))

### Normalize Data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
train_dataframe[molecular_properties] = scaler.fit_transform(train_dataframe[molecular_properties])
test_dataframe[molecular_properties] = scaler.transform(test_dataframe[molecular_properties])
validation_dataframe[molecular_properties] = scaler.transform(validation_dataframe[molecular_properties])

# scaler.inverse_transform(train_dataframe[molecular_property])

In [ ]:
train_dataframe

### Tokenize Data

In [ ]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_dataframe),
    'test': Dataset.from_pandas(test_dataframe),
    'validation': Dataset.from_pandas(validation_dataframe)
})

dataset

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.model_max_len=512

In [ ]:
def tokenize(batch):
  return tokenizer(batch["ecfp1"], truncation=True, max_length=512, padding='max_length')

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

In [ ]:
columns = ["input_ids", "attention_mask"]
columns.extend(molecular_properties) # our labels
print(columns)
tokenized_dataset.set_format('torch', columns=columns)

### Create Model

In [ ]:
from transformers import AutoModel, AutoConfig

class MolecularPropertiesRegression(torch.nn.Module):
    def __init__(self, model_name, num_properties):
        super(MolecularPropertiesRegression, self).__init__()
        self.num_properties = num_properties

        config = AutoConfig.from_pretrained(model_name)
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        # removing last layer of transformer
        self.transformer.pooler = torch.nn.Identity()
        # freezing transformer weights
        for param in self.transformer.parameters():
            param.requires_grad = False
        self.regressor = torch.nn.Linear(768, num_properties)

    def forward(self, input_ids = None, attention_mask=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
        last_hidden_state = outputs[0]
        # last_hidden_state is the shape of (batch_size=32, input_sequence_length=512, hidden_size=768)
        # so we take only hidden emdedding for [CLS] token (first) as it contains the entire context
        # and would be sufficient for simple downstream tasks such as classification/regression
        predicted_property_values = self.regressor(last_hidden_state[:, 0, : ].view(-1, 768))

        return predicted_property_values
        

### Create PyTorch DataLoader

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_dataset['train'], shuffle = True, batch_size = 1024, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset['validation'], shuffle = True, batch_size = 512, collate_fn = data_collator
)

In [ ]:
device = torch.device("cuda", index=2) if torch.cuda.is_available() else torch.device('cpu')

model = MolecularPropertiesRegression(model_name, num_properties=len(molecular_properties)).to(device)

In [ ]:
model

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epoch = 3

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

loss_func = torch.nn.MSELoss()

In [ ]:
from datasets import load_metric

# a metric for each property
metrics = { k: load_metric("mse") for k in molecular_properties }

### Training

In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))


for epoch in range(num_epoch):
    model.train()
    for batch in train_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        labels_batch = { k: v.to(device) for k, v in batch.items() if k in molecular_properties }

        labeled_property_values = torch.stack(list(labels_batch.values())).T
        predicted_property_values = model(**input_batch)
        
        loss = loss_func(predicted_property_values, labeled_property_values)
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    model.eval()
    for batch in eval_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        labels_batch = { k: v.to(device) for k, v in batch.items() if k in molecular_properties }

        labeled_property_values = torch.stack(list(labels_batch.values())).T
        with torch.no_grad():
            predicted_property_values = model(**input_batch)

        for i, molecular_property in enumerate(molecular_properties):
            metrics[molecular_property].add_batch(predictions = [predicted_property_values[0][i]], references = [labeled_property_values[0][i]])
        progress_bar_eval.update(1)
    
    for molecular_property in molecular_properties:
        print(f'Metric for {molecular_property}:', metrics[molecular_property].compute()) 

## Post Training Evaluation

In [ ]:
model.eval()

test_dataloader = DataLoader(
    tokenized_dataset['test'], batch_size = 512, collate_fn = data_collator
)

for batch in tqdm(test_dataloader):
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        labels_batch = { k: v.to(device) for k, v in batch.items() if k in molecular_properties }

        labeled_property_values = torch.stack(list(labels_batch.values())).T
        with torch.no_grad():
            predicted_property_values = model(**input_batch)

        for i, molecular_property in enumerate(molecular_properties):
            metrics[molecular_property].add_batch(predictions = [predicted_property_values[0][i]], references = [labeled_property_values[0][i]])
    
for molecular_property in molecular_properties:
    print(f'Metric for {molecular_property}:', metrics[molecular_property].compute())

In [ ]:
torch.save(model, model_name + '_and_linear_3epochs.pt')

In [ ]:
torch.cuda.empty_cache()